# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.


In [1]:
import torch
from torch import nn

(**We start by focusing on an MLP with one hidden layer.**)


In [88]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.


When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.


We can inspect the parameters of the second fully connected layer as follows.


In [89]:
net[0].state_dict()

OrderedDict([('weight',
              tensor([[-0.1709, -0.1406, -0.4279, -0.1338],
                      [ 0.2216,  0.2282, -0.4397, -0.2073],
                      [-0.4880,  0.2239, -0.0268, -0.2185],
                      [-0.0167, -0.2703, -0.1663, -0.4087],
                      [ 0.4300, -0.3129,  0.4477,  0.0756],
                      [-0.3256, -0.2088,  0.3990,  0.1338],
                      [-0.1164, -0.1817,  0.2940,  0.3005],
                      [-0.3111, -0.4293,  0.4876,  0.0462]])),
             ('bias',
              tensor([ 0.2620,  0.0964,  0.1856, -0.3303,  0.3859, -0.1541,  0.2219, -0.0893]))])

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.


In [ ]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0709]))

In [14]:
net[2].weight.data[0]

tensor([ 0.1828,  0.1423, -0.0567, -0.1793,  0.3061, -0.0629, -0.3294, -0.2392])

Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.


In [16]:
net[2].weight.grad == None

True

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.


In [17]:
[(name, param) for name, param in net.named_parameters()]

[('0.weight',
  Parameter containing:
  tensor([[ 0.1334, -0.1237, -0.3839, -0.2493],
          [-0.2873, -0.1194, -0.1950,  0.2753],
          [ 0.4725,  0.4304,  0.0616, -0.2288],
          [-0.2622, -0.3297,  0.3641, -0.2552],
          [ 0.2952, -0.3730, -0.2026,  0.2635],
          [ 0.4092,  0.1098,  0.1583,  0.2103],
          [-0.3041,  0.1532,  0.1656, -0.2515],
          [-0.0381,  0.2765, -0.3207, -0.2442]], requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([-0.2282,  0.3261, -0.4351,  0.3788,  0.3820,  0.0589,  0.0012,  0.3643],
         requires_grad=True)),
 ('2.weight',
  Parameter containing:
  tensor([[ 0.1828,  0.1423, -0.0567, -0.1793,  0.3061, -0.0629, -0.3294, -0.2392]],
         requires_grad=True)),
 ('2.bias',
  Parameter containing:
  tensor([-0.0622], requires_grad=True))]

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.


In [22]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2].weight.data[0, 0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0, 0])
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

ReLU()
tensor([True, True, True, True, True, True, True, True])
tensor(0.0584)
tensor(100.)
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.


You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.


## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


In [45]:
#Excercise 1

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

nest = NestMLP()
nest(X)

#Access all parameters in the first two hidden layers
[(name, param.shape) for name, param in nest.net.named_parameters()]

#Access weights of the first hidden layer
nest.net[0].weight.data

#Access all parameters of the last hidden layer
nest.linear.state_dict()

torch.Size([2, 16])


In [90]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

#Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.

shared = nn.LazyLinear(4)
net = nn.Sequential(nn.LazyLinear(4), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

# Generate some sample data
X = torch.randn(10, 4)
y = torch.randn(10, 1)

# Create a dataset and data loader
dataset = TensorDataset(X, y)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(data_loader):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = net(data)

        # Calculate the loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Print the parameters and gradients for the shared layer (optional)
        if batch_idx % 10 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}")
            for name, param in shared.named_parameters():
                print(f"Parameter: {name}, Value: {param.data}")
                if param.grad is not None:
                    print(f"Gradient: {name}, Value: {param.grad}")

Epoch: 0, Batch: 0
Parameter: weight, Value: tensor([[ 0.3454, -0.2830, -0.1320, -0.1220],
        [-0.0330,  0.2569, -0.3630, -0.3798],
        [-0.4470, -0.4838,  0.1064, -0.1026],
        [ 0.2153,  0.3756,  0.0189,  0.2346]])
Gradient: weight, Value: tensor([[-2.6510e-03, -6.6914e-04, -6.1215e-03, -4.8917e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.7456e-05,  0.0000e+00],
        [-2.2884e-04,  3.2236e-04, -3.9631e-04,  2.9306e-04]])
Parameter: bias, Value: tensor([ 0.2009, -0.0364, -0.0035, -0.2945])
Gradient: bias, Value: tensor([-0.0217,  0.0000,  0.0004,  0.0002])
Epoch: 1, Batch: 0
Parameter: weight, Value: tensor([[ 0.3454, -0.2830, -0.1320, -0.1220],
        [-0.0330,  0.2569, -0.3630, -0.3798],
        [-0.4470, -0.4838,  0.1064, -0.1026],
        [ 0.2153,  0.3756,  0.0189,  0.2346]])
Gradient: weight, Value: tensor([[-2.7391e-03, -7.0578e-04, -6.4194e-03, -4.8846e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000

[Discussions](https://discuss.d2l.ai/t/57)
